In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime
import time
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
train = pd.read_csv("../input/g-research-crypto-forecasting/train.csv",
                              dtype = {'Asset_ID' : 'int8', 'Count' : 'float16', 'Open' : 'float32', 
                                       'High' : 'float32', 'Low' : 'float32', 'Close' : 'float32', 
                                       'Volume' : 'float16', 'VWAP' : 'float16', 'Target' : 'float16'})

asset_details = pd.read_csv("../input/g-research-crypto-forecasting/asset_details.csv")

In [3]:
train_copy = train.copy().set_index("timestamp")


##Bitcoin
train_BTC = train_copy[train_copy['Asset_ID']==1]

##Dogecoin
train_DOGE = train_copy[train_copy['Asset_ID']==4]

##Etherium (Ether)
train_ETH = train_copy[train_copy['Asset_ID']==6]

##Litecoin
train_LTC = train_copy[train_copy['Asset_ID']==9]

Asset_list = [train_BTC, train_DOGE, train_ETH, train_LTC]

In [4]:
##Imputing missing timestamps
train_BTC = train_BTC.reindex(range(train_BTC.index[0], train_BTC.index[-1]+60, 60), method='pad')
train_ETH = train_ETH.reindex(range(train_ETH.index[0], train_ETH.index[-1]+60, 60), method='pad')
train_LTC = train_LTC.reindex(range(train_LTC.index[0], train_LTC.index[-1]+60, 60), method='pad')
train_DOGE = train_DOGE.reindex(range(train_DOGE.index[0], train_DOGE.index[-1]+60, 60), method='pad')


##Imputing the missing values with 0
train_BTC.Target.fillna(0,inplace=True)
train_ETH.Target.fillna(0,inplace=True)
train_LTC.Target.fillna(0,inplace=True)
train_DOGE.Target.fillna(0,inplace=True)


## Function to calculate log returns
def log_return(series, periods=1):
    return np.log(series).diff(periods=periods)

## Functions to calculate upper and lower shadow
upper_shadow = lambda asset: asset.High - np.maximum(asset.Close, asset.Open)
lower_shadow = lambda asset: np.minimum(asset.Close, asset.Open) - asset.Low

X_BTC = pd.concat([log_return(train_BTC.VWAP, periods=5), log_return(train_BTC.VWAP, periods=5).abs(), 
                   upper_shadow(train_BTC), lower_shadow(train_BTC)], axis=1)
y_BTC = train_BTC.Target

X_ETH = pd.concat([log_return(train_ETH.VWAP, periods=5), log_return(train_ETH.VWAP, periods=5).abs(), 
                   upper_shadow(train_ETH), lower_shadow(train_ETH)], axis=1)
y_ETH = train_ETH.Target

X_LTC = pd.concat([log_return(train_LTC.VWAP, periods=5), log_return(train_LTC.VWAP, periods=5).abs(), 
                   upper_shadow(train_LTC), lower_shadow(train_LTC)], axis=1)
y_LTC = train_LTC.Target

X_DOGE = pd.concat([log_return(train_DOGE.VWAP, periods=5), log_return(train_DOGE.VWAP, periods=5).abs(), 
                   upper_shadow(train_DOGE), lower_shadow(train_DOGE)], axis=1)
y_DOGE = train_DOGE.Target

In [5]:
## selecting training and testing windows

totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, '%d/%m/%Y').timetuple()))

train_window = [totimestamp("01/05/2021"), totimestamp("31/05/2021")]
test_window = [totimestamp("01/06/2021"), totimestamp("30/06/2021")]


X_BTC_train = X_BTC.loc[train_window[0]:train_window[1]].fillna(0).to_numpy()  # filling NaN's with zeros
y_BTC_train = y_BTC.loc[train_window[0]:train_window[1]].fillna(0).to_numpy()

X_BTC_test = X_BTC.loc[test_window[0]:test_window[1]].fillna(0).to_numpy()
y_BTC_test = y_BTC.loc[test_window[0]:test_window[1]].fillna(0).to_numpy()

X_ETH_train = X_ETH.loc[train_window[0]:train_window[1]].fillna(0).to_numpy()
y_ETH_train = y_ETH.loc[train_window[0]:train_window[1]].fillna(0).to_numpy()

X_ETH_test = X_ETH.loc[test_window[0]:test_window[1]].fillna(0).to_numpy()
y_ETH_test = y_ETH.loc[test_window[0]:test_window[1]].fillna(0).to_numpy()

X_LTC_train = X_LTC.loc[train_window[0]:train_window[1]].fillna(0).to_numpy()
y_LTC_train = y_LTC.loc[train_window[0]:train_window[1]].fillna(0).to_numpy() 

X_LTC_test = X_LTC.loc[test_window[0]:test_window[1]].fillna(0).to_numpy()
y_LTC_test = y_LTC.loc[test_window[0]:test_window[1]].fillna(0).to_numpy()

X_DOGE_train = X_DOGE.loc[train_window[0]:train_window[1]].fillna(0).to_numpy()
y_DOGE_train = y_DOGE.loc[train_window[0]:train_window[1]].fillna(0).to_numpy() 

X_DOGE_test = X_DOGE.loc[test_window[0]:test_window[1]].fillna(0).to_numpy() 
y_DOGE_test = y_DOGE.loc[test_window[0]:test_window[1]].fillna(0).to_numpy()

In [6]:
scaler = StandardScaler()

X_BTC_train_scaled = scaler.fit_transform(X_BTC_train)
X_BTC_test_scaled = scaler.transform(X_BTC_test)

X_ETH_train_scaled = scaler.fit_transform(X_ETH_train)
X_ETH_test_scaled = scaler.transform(X_ETH_test)

X_LTC_train_scaled = scaler.fit_transform(X_LTC_train)
X_LTC_test_scaled = scaler.transform(X_LTC_test)

X_DOGE_train_scaled = scaler.fit_transform(X_DOGE_train)
X_DOGE_test_scaled = scaler.transform(X_DOGE_test)